## Descarga de datos Meteorológicos - Senamhi

![Descarga de datos Meteorológicos](image.png)

In [2]:
!where python

C:\Users\ASUS\.conda\envs\jonatan\python.exe
C:\Users\ASUS\anaconda3\python.exe
C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\python.exe


In [3]:
import sys
print(sys.executable)

C:\Users\ASUS\.conda\envs\jonatan\python.exe


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
#from matplotlib import rc
#rc('font', **{'family': 'serif', 'serif': ['Times New Roman']},size=12)# 'Computer Modern'
#rc('text', usetex=False)

In [4]:
########################################## Download stations of Senamhi ############################

g = open("D:/S/SENAMHI PERU/San Martin/NARANJILLO.txt", 'r')

year = [] # year
month = [] # month
day = []   # day
J1 = [] # prec
J2 = [] # tmax
J3 = [] # tmin

for line in g:
    s = line[:].split()
    year.append(float(s[0]))
    month.append(float(s[1]))
    day.append(float(s[2]))
    J1.append(float(s[3]))
    J2.append(float(s[4]))
    J3.append(float(s[5]))
      
data = pd.DataFrame({'año': year,
                     'mes': month, 
                     'dia': day, 
                     'prec': J1, 
                     'tmax': J2,
                     'tmin': J3}) 

In [7]:
data = pd.DataFrame({
    'fecha': pd.to_datetime({'year': year, 'month': month, 'day': day}),
    'prec': J1,
    'tmax': J2,
    'tmin': J3
})
data.set_index('fecha', inplace=True)


In [9]:
#Crear el rango de fechas deseado: 1981-01-01 al 2015-12-31 ===
rango_completo = pd.date_range(start='1981-01-01', end='2015-12-31', freq='D')
rango_completo

DatetimeIndex(['1981-01-01', '1981-01-02', '1981-01-03', '1981-01-04',
               '1981-01-05', '1981-01-06', '1981-01-07', '1981-01-08',
               '1981-01-09', '1981-01-10',
               ...
               '2015-12-22', '2015-12-23', '2015-12-24', '2015-12-25',
               '2015-12-26', '2015-12-27', '2015-12-28', '2015-12-29',
               '2015-12-30', '2015-12-31'],
              dtype='datetime64[ns]', length=12783, freq='D')

In [11]:
# === 4. Reindexar el DataFrame para cubrir todo el rango ===
data_completa = data.reindex(rango_completo)
data_completa

,prec,tmax,tmin
1981-01-01,0.0,-99.9,-99.9
1981-01-02,3.0,-99.9,-99.9
1981-01-03,2.0,-99.9,-99.9
1981-01-04,0.0,-99.9,-99.9
1981-01-05,0.0,-99.9,-99.9
...,...,...,...
2015-12-27,NaN,NaN,NaN
2015-12-28,NaN,NaN,NaN
2015-12-29,NaN,NaN,NaN
2015-12-30,NaN,NaN,NaN


In [13]:
# === 5. Rellenar valores faltantes con -99.9 ===
data_completa.fillna(-99.9, inplace=True)
data_completa

,prec,tmax,tmin
1981-01-01,0.0,-99.9,-99.9
1981-01-02,3.0,-99.9,-99.9
1981-01-03,2.0,-99.9,-99.9
1981-01-04,0.0,-99.9,-99.9
1981-01-05,0.0,-99.9,-99.9
...,...,...,...
2015-12-27,-99.9,-99.9,-99.9
2015-12-28,-99.9,-99.9,-99.9
2015-12-29,-99.9,-99.9,-99.9
2015-12-30,-99.9,-99.9,-99.9


In [14]:
# === 6. Renombrar el índice si quieres llamarlo 'fecha' ===
data_completa.index.name = 'fecha'

# === 7. Mostrar los primeros días para verificar ===
print(data_completa.head())

            prec  tmax  tmin
fecha                       
1981-01-01   0.0 -99.9 -99.9
1981-01-02   3.0 -99.9 -99.9
1981-01-03   2.0 -99.9 -99.9
1981-01-04   0.0 -99.9 -99.9
1981-01-05   0.0 -99.9 -99.9


In [19]:
import pandas as pd

def process_station(file_path, start='1981-01-01', end='2015-12-31'):
    # Create target date range
    date_range = pd.date_range(start=start, end=end, freq='D')
    final_df = pd.DataFrame(index=date_range)
    final_df.index.name = 'date'
    final_df['prec'] = -99.9
    final_df['tmax'] = -99.9
    final_df['tmin'] = -99.9

    # Read station file
    with open(file_path, 'r') as f:
        years, months, days = [], [], []
        precs, tmaxs, tmins = [], [], []
        for line in f:
            parts = line.strip().split()
            years.append(int(parts[0]))
            months.append(int(parts[1]))
            days.append(int(parts[2]))
            precs.append(float(parts[3]))
            tmaxs.append(float(parts[4]))
            tmins.append(float(parts[5]))
    
    # Create original DataFrame
    data = pd.DataFrame({
        'year': years,
        'month': months,
        'day': days,
        'prec': precs,
        'tmax': tmaxs,
        'tmin': tmins
    })

    # Combine into datetime and set index
    data['date'] = pd.to_datetime(data[['year', 'month', 'day']])
    data = data.set_index('date')
    data = data[['prec', 'tmax', 'tmin']]

    # Fill in final DataFrame
    final_df.update(data)

    return final_df


In [6]:
################################################# Download Senamhi stations for a defined time #########
import os
import pandas as pd

# Tu función process_station
def process_station(file_path, start='1981-01-01', end='2015-12-31'): # time
    date_range = pd.date_range(start=start, end=end, freq='D')
    final_df = pd.DataFrame(index=date_range)
    final_df.index.name = 'date'
    final_df['prec'] = -99.9
    final_df['tmax'] = -99.9
    final_df['tmin'] = -99.9

    with open(file_path, 'r') as f:
        years, months, days = [], [], []
        precs, tmaxs, tmins = [], [], []
        for line in f:
            parts = line.strip().split()
            years.append(int(parts[0]))
            months.append(int(parts[1]))
            days.append(int(parts[2]))
            precs.append(float(parts[3]))
            tmaxs.append(float(parts[4]))
            tmins.append(float(parts[5]))

    data = pd.DataFrame({
        'year': years,
        'month': months,
        'day': days,
        'prec': precs,
        'tmax': tmaxs,
        'tmin': tmins
    })

    data['date'] = pd.to_datetime(data[['year', 'month', 'day']])
    data = data.set_index('date')
    data = data[['prec', 'tmax', 'tmin']]
    final_df.update(data)

    return final_df

# === Files ===
input_root = "D:/S/SENAMHI PERU"
output_root = "D:/S//OUTPUT_CSV_SENAMHI"  # Puedes cambiar a donde quieras guardar

# === Recorrer todos los departamentos y estaciones ===
for dept in os.listdir(input_root):
    dept_path = os.path.join(input_root, dept)
    if os.path.isdir(dept_path):
        for filename in os.listdir(dept_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(dept_path, filename)
                print(f"Procesando: {file_path}")

                # Procesar estación
                df = process_station(file_path)

                # Crear carpeta de salida si no existe
                output_dir = os.path.join(output_root, dept)
                os.makedirs(output_dir, exist_ok=True)

                # Guardar CSV con mismo nombre de estación
                output_file = os.path.join(output_dir, filename.replace('.txt', '.csv'))
                df.to_csv(output_file)


Procesando: D:/S/SENAMHI PERU\Amazonas\EL PINTOR.txt
Procesando: D:/S/SENAMHI PERU\Amazonas\MAGUNCHAL.txt
Procesando: D:/S/SENAMHI PERU\Ancash\BUENA VISTA.txt
Procesando: D:/S/SENAMHI PERU\Ancash\CHACCHAN.txt
Procesando: D:/S/SENAMHI PERU\Ancash\MAYORARCA.txt
Procesando: D:/S/SENAMHI PERU\Ancash\MILPO.txt
Procesando: D:/S/SENAMHI PERU\Ancash\OCROS.txt
Procesando: D:/S/SENAMHI PERU\Ancash\PIRA.txt
Procesando: D:/S/SENAMHI PERU\Ancash\RECUAY.txt
Procesando: D:/S/SENAMHI PERU\Ancash\SIHUAS.txt
Procesando: D:/S/SENAMHI PERU\Apurimac\CURAHUASI.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\ANDAHUA.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\APLAO.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\AYO.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CARAVELI.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHACHAS.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHICHAS.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHIGUATA.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHIVAY.txt
Procesando: D:/S/SENAMHI PERU\Arequipa\CHOCO.txt

In [7]:
######################################        Download PISCO data       ####################################
import xarray as xr #   ! conda install -c conda-forge pydap
import pandas as pd
# Specify the longitude and latitude range for which we want to get the IRI forecast
# Bounding box que cubre todo Perú
longitudes = ['-81.35', '-68.65']  # Longitudes de Perú (Oeste a Este)
latitudes = ['-18.35', '-0.05']    # Latitudes de Perú (Sur a Norte)

# Construct the IRI data URL with specified longitude and latitude range
url = f"""https://iridl.ldeo.columbia.edu/SOURCES/.SENAMHI/.HSR/.PISCO/.Prec/\
.v2p1/.stable/.daily/.Prec/\
X/{longitudes[0]}/{longitudes[1]}/RANGEEDGES/\
Y/{latitudes[0]}/{latitudes[1]}/RANGEEDGES/\
dods"""

# Open the dataset using xarray
dataset = xr.open_dataset(url, decode_times=False)
#dataset['T'].attrs['calendar'] = '360_day'; # of month

# Decode time information (if not already decoded)
dataset = xr.decode_cf(dataset, decode_times=True)

# Rename dimensions and variables to more descriptive names
dataset = dataset.rename({
    'T': 'time',       # (1 Jan 1981) to (31 Dec 2016) by 1.0 N= 13149 pts :grid
    'X': 'lon',        # Longitude (81.25W) to (68.05W) by 0.09999998 N= 133 pts :grid
    'Y': 'lat'         # Latitude  (0.95N) to (18.75S) by 0.1 N= 198 pts :grid
})

# Optional: If lead time values are offset by 0.5, this could round them to integers
# dataset['lead'] = (dataset['lead'] - 0.5).astype('int')

# Convert the time values from string to pandas datetime
dataset['time'] = pd.to_datetime(dataset['time'].values.astype(str), yearfirst=True)

# At this point, dataset is ready for analysis or visualization


C:\Users\ASUS\.conda\envs\jonatan\lib\site-packages\xarray\backends\plugins.py:64: RuntimeWarning: Engine 'rasterio' loading failed:
DLL load failed while importing _base: No se puede encontrar el módulo especificado.
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [8]:
dataset # conda install -c conda-forge xarray netCDF4 pydap

<xarray.Dataset>
Dimensions:  (time: 13149, lon: 127, lat: 184)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T12:00:00 ... 2016-12-31T12:00:00
  * lon      (lon) float32 -81.25 -81.15 -81.05 -80.95 ... -68.85 -68.75 -68.65
  * lat      (lat) float32 -18.35 -18.25 -18.15 -18.05 ... -0.25 -0.15 -0.05
Data variables:
    Prec     (time, lat, lon) float32 ...
Attributes:
    Conventions:  IRIDL

In [14]:
#import hvplot #       ! conda install -c conda-forge hvplot  # #!pip install polars
#dataset['Prec'].isel(time=0).hvplot.image(x='lon', y='lat', cmap='jet')

In [1]:
import xarray as xr ########  Download prec, tmax and tmin of grilled data PISCO 
import pandas as pd

# === Rango de coordenadas para todo el Perú ===
longitudes = ['-81.35', '-68.65']  # Oeste a Este
latitudes  = ['-18.35', '-0.05']   # Sur a Norte

# === Función para abrir y procesar un dataset PISCO ===
def load_pisco_dataset(var_url, var_name):
    url = f"""{var_url}\
X/{longitudes[0]}/{longitudes[1]}/RANGEEDGES/\
Y/{latitudes[0]}/{latitudes[1]}/RANGEEDGES/\
dods"""

    ds = xr.open_dataset(url, decode_times=False)
    ds = xr.decode_cf(ds, decode_times=True)
    ds = ds.rename({
        'T': 'time',
        'X': 'lon',
        'Y': 'lat'
    })
    ds['time'] = pd.to_datetime(ds['time'].values.astype(str), yearfirst=True)

    # Cambiar el nombre de la variable interna al que queremos
    var_original = list(ds.data_vars)[0]
    ds = ds.rename({var_original: var_name})

    return ds

# === URLs base para cada variable ===
url_prec_base = "https://iridl.ldeo.columbia.edu/SOURCES/.SENAMHI/.HSR/.PISCO/.Prec/.v2p1/.stable/.daily/.Prec/"
url_tmax_base = "https://iridl.ldeo.columbia.edu/SOURCES/.SENAMHI/.HSR/.PISCO/.Temp/.v1p1/.tmax/.stable/.daily/.tmax/"
url_tmin_base = "https://iridl.ldeo.columbia.edu/SOURCES/.SENAMHI/.HSR/.PISCO/.Temp/.v1p1/.tmin/.stable/.daily/.tmin/"

# === Cargar datasets ===
print("Cargando precipitación...")
ds_prec = load_pisco_dataset(url_prec_base, "prec")

print("Cargando Tmax...")
ds_tmax = load_pisco_dataset(url_tmax_base, "tmax")

print("Cargando Tmin...")
ds_tmin = load_pisco_dataset(url_tmin_base, "tmin")

# === Ver en Jupyter antes de guardar ===
print("\n--- Precipitación ---")
display(ds_prec)

print("\n--- Tmax ---")
display(ds_tmax)

print("\n--- Tmin ---")
display(ds_tmin)

# === Combinar en un solo Dataset ===
#ds_all = xr.merge([ds_prec, ds_tmax, ds_tmin]) # of small area

#print("\n--- Dataset combinado ---")
#display(ds_all)

# Si luego quieres guardar:
# ds_all.to_netcdf("pisco_peru.nc")

Cargando precipitación...


C:\Users\ASUS\.conda\envs\jonatan\lib\site-packages\xarray\backends\plugins.py:64: RuntimeWarning: Engine 'rasterio' loading failed:
DLL load failed while importing _base: No se puede encontrar el módulo especificado.
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


Cargando Tmax...
Cargando Tmin...

--- Precipitación ---


<xarray.Dataset>
Dimensions:  (time: 13149, lon: 127, lat: 184)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T12:00:00 ... 2016-12-31T12:00:00
  * lon      (lon) float32 -81.25 -81.15 -81.05 -80.95 ... -68.85 -68.75 -68.65
  * lat      (lat) float32 -18.35 -18.25 -18.15 -18.05 ... -0.25 -0.15 -0.05
Data variables:
    prec     (time, lat, lon) float32 ...
Attributes:
    Conventions:  IRIDL


--- Tmax ---


<xarray.Dataset>
Dimensions:  (time: 13149, lon: 128, lat: 184)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T12:00:00 ... 2016-12-31T12:00:00
  * lon      (lon) float32 -81.35 -81.25 -81.15 -81.05 ... -68.85 -68.75 -68.65
  * lat      (lat) float32 -18.35 -18.25 -18.15 -18.05 ... -0.25 -0.15 -0.05
Data variables:
    tmax     (time, lat, lon) float32 ...
Attributes:
    Conventions:  IRIDL


--- Tmin ---


<xarray.Dataset>
Dimensions:  (time: 13149, lon: 128, lat: 184)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T12:00:00 ... 2016-12-31T12:00:00
  * lon      (lon) float32 -81.35 -81.25 -81.15 -81.05 ... -68.85 -68.75 -68.65
  * lat      (lat) float32 -18.35 -18.25 -18.15 -18.05 ... -0.25 -0.15 -0.05
Data variables:
    tmin     (time, lat, lon) float32 ...
Attributes:
    Conventions:  IRIDL

In [5]:
import polars as pl  #!pip install polars

# Try reading with Latin-1 encoding (Windows default)
ubicaciones_df = pl.read_csv(
    "D:/S/SENAMHI PERU/Stations Senamhi.csv", 
    encoding="latin1"
)[["Stations", "Lat", "Lon"]]

# Convert to list of dictionaries
locs = ubicaciones_df.to_dicts()

# Preview the first few locations
print(locs[:5])

[{'Stations': 'CABANILLAS', 'Lat': -15.16, 'Lon': -70.0}, {'Stations': 'CAÑETE', 'Lat': -13.07, 'Lon': -76.3}, {'Stations': 'CHOTANO LAJAS', 'Lat': -6.56, 'Lon': -78.7}, {'Stations': 'LA CASCARILLA', 'Lat': -5.66, 'Lon': -78.9}, {'Stations': 'ÑAÑA', 'Lat': -11.99, 'Lon': -76.9}]


In [6]:
# === Supongamos que ya tienes cargados ds_prec, ds_tmax, ds_tmin ===
prec = xr.Dataset()
tmax = xr.Dataset()
tmin = xr.Dataset()

for l in locs:
    nombre = l['Stations']
    lon = l['Lon']
    lat = l['Lat']

    # --- Precipitación ---
    ds_prec_sel = ds_prec.sel(lon=lon, lat=lat, method='nearest')
    ds_prec_sel.attrs[f"{nombre}_lon"] = float(ds_prec_sel.lon.values)
    ds_prec_sel.attrs[f"{nombre}_lat"] = float(ds_prec_sel.lat.values)
    ds_prec_sel = ds_prec_sel.rename({"prec": nombre}).drop_vars(["lat", "lon"])
    prec = xr.merge([prec, ds_prec_sel], compat="override")

    # --- Tmax ---
    ds_tmax_sel = ds_tmax.sel(lon=lon, lat=lat, method='nearest')
    ds_tmax_sel.attrs[f"{nombre}_lon"] = float(ds_tmax_sel.lon.values)
    ds_tmax_sel.attrs[f"{nombre}_lat"] = float(ds_tmax_sel.lat.values)
    ds_tmax_sel = ds_tmax_sel.rename({"tmax": nombre}).drop_vars(["lat", "lon"])
    tmax = xr.merge([tmax, ds_tmax_sel], compat="override")

    # --- Tmin ---
    ds_tmin_sel = ds_tmin.sel(lon=lon, lat=lat, method='nearest')
    ds_tmin_sel.attrs[f"{nombre}_lon"] = float(ds_tmin_sel.lon.values)
    ds_tmin_sel.attrs[f"{nombre}_lat"] = float(ds_tmin_sel.lat.values)
    ds_tmin_sel = ds_tmin_sel.rename({"tmin": nombre}).drop_vars(["lat", "lon"])
    tmin = xr.merge([tmin, ds_tmin_sel], compat="override")

# Si luego quieres guardar comprimido:
# prec.to_netcdf("prec_estaciones.nc", encoding={v: {"zlib": True, "complevel": 4} for v in prec.data_vars})
prec.data_vars

Data variables:
    CABANILLAS                  (time) float32 ...
    CAÑETE                      (time) float32 ...
    CHOTANO LAJAS               (time) float32 ...
    LA CASCARILLA               (time) float32 ...
    ÑAÑA                        (time) float32 ...
    HUANGACOCHA                 (time) float32 ...
    AYAVIRI-PUN                 (time) float32 ...
    MUÑANI                      (time) float32 ...
    FRANCISCO DE ORELLANA       (time) float32 ...
    MANITI                      (time) float32 ...
    BELLAVISTA                  (time) float32 ...
    CHILCAYOC                   (time) float32 ...
    PAUZA                       (time) float32 ...
    PAMPA BLANCA-ARE            (time) float32 ...
    CAPACHICA                   (time) float32 ...
    PAMPAHUTA                   (time) float32 ...
    LAMPA                       (time) float32 ...
    CARAVELI                    (time) float32 ...
    YANAQUIHUA                  (time) float32 ...
    LAMPA-AYA  

In [8]:
import pandas as pd     #### it may take a few minutes

# === Diccionario final ===
clima_df_dict = {}

# Crear diccionarios base
prec_df_dict = {}
tmax_df_dict = {}
tmin_df_dict = {}

# --- Precipitación ---
for var_name in prec.data_vars:
    serie = prec[var_name].to_series()
    df = serie.to_frame(name='prec')
    prec_df_dict[var_name.strip()] = df

# --- Tmax ---
for var_name in tmax.data_vars:
    serie = tmax[var_name].to_series()
    df = serie.to_frame(name='tmax')
    tmax_df_dict[var_name.strip()] = df

# --- Tmin ---
for var_name in tmin.data_vars:
    serie = tmin[var_name].to_series()
    df = serie.to_frame(name='tmin')
    tmin_df_dict[var_name.strip()] = df

# --- Unir los tres en un solo diccionario ---
for estacion in prec_df_dict.keys():
    df_prec = prec_df_dict[estacion]
    df_tmax = tmax_df_dict.get(estacion, pd.DataFrame())
    df_tmin = tmin_df_dict.get(estacion, pd.DataFrame())

    # Unir por índice (tiempo)
    df_merged = df_prec.join(df_tmax, how="outer").join(df_tmin, how="outer")

    clima_df_dict[estacion] = df_merged

# clima_df_dict ahora tiene un DataFrame por estación con columnas: prec, tmax, tmin
#clima_df_dict['NOMBRE_ESTACION']


In [12]:
clima_df_dict['LOMAS DE LACHAY']

,prec,tmax,tmin
time,,,
1981-01-01 12:00:00,0.006526,24.741425,14.449268
1981-01-02 12:00:00,0.018746,25.795872,14.790736
1981-01-03 12:00:00,0.000000,24.870714,15.788437
1981-01-04 12:00:00,0.000000,24.007114,15.993378
1981-01-05 12:00:00,0.000000,24.188253,16.062469
...,...,...,...
2016-12-27 12:00:00,0.000000,23.773602,15.141025
2016-12-28 12:00:00,0.046240,23.093645,16.404392
2016-12-29 12:00:00,0.000454,23.596045,16.220774


In [17]:
################################# Complete the missing data Senamhi with PISCO data (prec) ##############################
import os
import pandas as pd
from pathlib import Path

# Ruta base de entrada y salida
input_base = Path("D:/S//OUTPUT_CSV_SENAMHI")
#output_base = Path("D:/S/OUTPUT_CSV_SENAMHI_NC")

# Asegurarse que las fechas de prec_df_dict estén en formato date (sin hora)
for key in prec_df_dict:
    df = prec_df_dict[key]
    df.index = df.index.normalize()
    prec_df_dict[key] = df

# Procesar cada archivo CSV
for dept_path in input_base.iterdir():
    if dept_path.is_dir():
        dept_name = dept_path.name
        for station_file in dept_path.glob("*.csv"):
            station_name = station_file.stem  # sin la extensión
            print(f"Procesando: {dept_name}/{station_name}")

            # Leer el CSV original
            df_csv = pd.read_csv(station_file, index_col=0, parse_dates=True)

            # Verificar si la estación existe en los datos de NetCDF
            if station_name in prec_df_dict:
                df_nc = prec_df_dict[station_name]

                # Asegurarse de que las fechas coincidan
                common_dates = df_csv.index.intersection(df_nc.index)

                # Reemplazar solo donde hay -99.9
                mask_replace = (df_csv.loc[common_dates, "prec"] == -99.9)
                df_csv.loc[common_dates, "prec"] = df_csv.loc[common_dates].apply(
                    lambda row: round(df_nc.loc[row.name, "prec"], 1) if row["prec"] == -99.9 else row["prec"],
                    axis=1
                )

                # Crear carpeta de salida si no existe
                output_dept_dir = output_base / dept_name
                output_dept_dir.mkdir(parents=True, exist_ok=True)

                # Guardar el nuevo archivo
                #output_path = output_dept_dir / f"{station_name}.csv"
                #df_csv.to_csv(output_path)
            else:
                print(f"⚠️ Estación no encontrada en NetCDF: {station_name}")


Procesando: Amazonas/EL PINTOR
Procesando: Amazonas/MAGUNCHAL
Procesando: Ancash/BUENA VISTA
Procesando: Ancash/CHACCHAN
Procesando: Ancash/MAYORARCA
Procesando: Ancash/MILPO
Procesando: Ancash/OCROS
Procesando: Ancash/PIRA
Procesando: Ancash/RECUAY
Procesando: Ancash/SIHUAS
Procesando: Apurimac/CURAHUASI
Procesando: Arequipa/ANDAHUA
Procesando: Arequipa/APLAO
Procesando: Arequipa/AYO
Procesando: Arequipa/CARAVELI
Procesando: Arequipa/CHACHAS
Procesando: Arequipa/CHICHAS
Procesando: Arequipa/CHIGUATA
Procesando: Arequipa/CHIVAY
Procesando: Arequipa/CHOCO
Procesando: Arequipa/COTAHUASI
Procesando: Arequipa/EL FRAYLE
Procesando: Arequipa/HUAMBO
Procesando: Arequipa/IMATA
Procesando: Arequipa/LA ANGOSTURA
Procesando: Arequipa/LA JOYA
Procesando: Arequipa/LA PAMPILLA
Procesando: Arequipa/LAS SALINAS
Procesando: Arequipa/MACHAGUAY
Procesando: Arequipa/MADRIGAL
Procesando: Arequipa/ORCOPAMPA
Procesando: Arequipa/PAMPA BLANCA-ARE
Procesando: Arequipa/PAMPA DE MAJES
Procesando: Arequipa/PILLON

In [22]:
################################# Complete the missing data Senamhi with PISCO data (prec, tmax, tmin) ##############################
import os
import pandas as pd
from pathlib import Path

# Rutas base
input_base = Path("D:/S/OUTPUT_CSV_SENAMHI")
output_base = Path("D:/S/OUTPUT_CSV_SENAMHI_NC")

# Normalizar fechas en diccionarios
for dic in (prec_df_dict, tmax_df_dict, tmin_df_dict):
    for key in dic:
        df = dic[key]
        df.index = df.index.normalize()
        dic[key] = df

# Procesar cada archivo CSV
for dept_path in input_base.iterdir():
    if dept_path.is_dir():
        dept_name = dept_path.name
        for station_file in dept_path.glob("*.csv"):
            station_name = station_file.stem
            print(f"Procesando: {dept_name}/{station_name}")

            # Leer CSV original
            df_csv = pd.read_csv(station_file, index_col=0, parse_dates=True)

            # Verificar si la estación existe en los datos de NetCDF
            if (
                station_name in prec_df_dict
                and station_name in tmax_df_dict
                and station_name in tmin_df_dict
            ):
                df_prec_nc = prec_df_dict[station_name]
                df_tmax_nc = tmax_df_dict[station_name]
                df_tmin_nc = tmin_df_dict[station_name]

                # Asegurarse de que las fechas coincidan
                common_dates = df_csv.index.intersection(df_prec_nc.index)

                # --- Reemplazar PREC ---
                df_csv.loc[common_dates, "prec"] = df_csv.loc[common_dates].apply(
                    lambda row: round(df_prec_nc.loc[row.name, "prec"], 1)
                    if row["prec"] == -99.9
                    else row["prec"],
                    axis=1
                )

                # --- Reemplazar TMAX ---
                if "tmax" in df_csv.columns:
                    df_csv.loc[common_dates, "tmax"] = df_csv.loc[common_dates].apply(
                        lambda row: round(df_tmax_nc.loc[row.name, "tmax"], 1)
                        if row["tmax"] == -99.9
                        else row["tmax"],
                        axis=1
                    )

                # --- Reemplazar TMIN ---
                if "tmin" in df_csv.columns:
                    df_csv.loc[common_dates, "tmin"] = df_csv.loc[common_dates].apply(
                        lambda row: round(df_tmin_nc.loc[row.name, "tmin"], 1)
                        if row["tmin"] == -99.9
                        else row["tmin"],
                        axis=1
                    )

                # Guardar en carpeta de salida
                output_dept_dir = output_base / dept_name
                output_dept_dir.mkdir(parents=True, exist_ok=True)
                output_path = output_dept_dir / f"{station_name}.csv"
                df_csv.to_csv(output_path)

            else:
                print(f"⚠️ Estación no encontrada en NetCDF: {station_name}")


Procesando: Amazonas/EL PINTOR
Procesando: Amazonas/MAGUNCHAL
Procesando: Ancash/BUENA VISTA
Procesando: Ancash/CHACCHAN
Procesando: Ancash/MAYORARCA
Procesando: Ancash/MILPO
Procesando: Ancash/OCROS
Procesando: Ancash/PIRA
Procesando: Ancash/RECUAY
Procesando: Ancash/SIHUAS
Procesando: Apurimac/CURAHUASI
Procesando: Arequipa/ANDAHUA
Procesando: Arequipa/APLAO
Procesando: Arequipa/AYO
Procesando: Arequipa/CARAVELI
Procesando: Arequipa/CHACHAS
Procesando: Arequipa/CHICHAS
Procesando: Arequipa/CHIGUATA
Procesando: Arequipa/CHIVAY
Procesando: Arequipa/CHOCO
Procesando: Arequipa/COTAHUASI
Procesando: Arequipa/EL FRAYLE
Procesando: Arequipa/HUAMBO
Procesando: Arequipa/IMATA
Procesando: Arequipa/LA ANGOSTURA
Procesando: Arequipa/LA JOYA
Procesando: Arequipa/LA PAMPILLA
Procesando: Arequipa/LAS SALINAS
Procesando: Arequipa/MACHAGUAY
Procesando: Arequipa/MADRIGAL
Procesando: Arequipa/ORCOPAMPA
Procesando: Arequipa/PAMPA BLANCA-ARE
Procesando: Arequipa/PAMPA DE MAJES
Procesando: Arequipa/PILLON

In [26]:
from pathlib import Path ############# of all files except some subfolders
import shutil

# Carpeta base con subcarpetas de departamentos
output_base = Path("D:/S/OUTPUT_CSV_SENAMHI_NC")

# Carpeta donde juntarás todos los CSV excepto los de una carpeta
consolidated_dir = Path("D:/S/OUTPUT_CSV_SENAMHI_NC_ALL")
consolidated_dir.mkdir(exist_ok=True)

# Nombre de la carpeta que quieres excluir
carpetas_excluir = {"Loreto", "San Martin"} # aut of study area

# Recorrer todas las subcarpetas y copiar archivos que NO sean de la carpeta excluida
for csv_file in output_base.rglob("*.csv"):
    # Obtener nombre de la subcarpeta (departamento)
    relative_parts = csv_file.relative_to(output_base).parts
    if relative_parts[0] in carpetas_excluir:
        continue # saltar esta carpeta

    # Copiar archivo
    destination = consolidated_dir / csv_file.name
    shutil.copy(csv_file, destination)

print(f"✅ Todos los CSV fueron consolidados en la carpeta OUTPUT_CSV_SENAMHI_NC_ALL (excepto '{carpetas_excluir}')")


✅ Todos los CSV fueron consolidados en la carpeta OUTPUT_CSV_SENAMHI_NC_ALL (excepto '{'San Martin', 'Loreto'}')


In [23]:
from pathlib import Path  ###############  for some subfolders except the others
import shutil

# Carpeta base con subcarpetas por departamento
output_base = Path("D:/S/OUTPUT_CSV_SENAMHI_NC")

# Carpeta destino donde guardarás los CSV seleccionados
consolidated_dir = Path("D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNOmore")
consolidated_dir.mkdir(parents=True, exist_ok=True)

# Subcarpetas que SÍ quieres incluir
carpetas_incluir = {"Puno"}  # usa el nombre exacto de las carpetas {"Cusco", "Puno"}  

# Copiar archivos de las carpetas seleccionadas, conservando el nombre original
for carpeta in carpetas_incluir:
    carpeta_path = output_base / carpeta
    if not carpeta_path.exists():
        print(f"⚠️ Carpeta no encontrada: {carpeta_path}")
        continue

    for csv_file in carpeta_path.glob("*.csv"):
        destino = consolidated_dir / csv_file.name  # usa el nombre original
        shutil.copy(csv_file, destino)
        #print(f"Copiado: {csv_file} → {destino}")

print("✅ Todos los CSV fueron copiados con su nombre original desde:", ", ".join(carpetas_incluir))



✅ Todos los CSV fueron copiados con su nombre original desde: Puno


In [33]:
import os ##########################  Wheather Interpolation-app ################################################
import pandas as pd
import unidecode
from glob import glob

# === 1. Cargar coordenadas ===
coord_df = pd.read_csv("D:/S/SENAMHI PERU/Stations Senamhi.csv", encoding="latin1")

# Limpiar nombres: quitar espacios, mayúsculas, tildes
coord_df["station_clean"] = coord_df["Stations"].str.strip().str.upper().apply(unidecode.unidecode)

# Crear diccionario para buscar coordenadas rápido
coord_dict = dict(zip(coord_df["station_clean"], zip(coord_df["Lat"], coord_df["Lon"])))

# === 2. Procesar archivos .csv ===
folder = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_ONLY"
all_files = glob(os.path.join(folder, "*.csv"))

merged_df = None
station_names = []
errores = []

for file in all_files:
    name = os.path.basename(file).replace(".csv", "")
    name_clean = unidecode.unidecode(name.strip().upper())

    if name_clean not in coord_dict:
        print(f"⚠️ Estación {name} no está en la tabla de coordenadas. Se ignora.")
        continue

    try:
        df = pd.read_csv(file)

        # Validar que tenga columnas mínimas esperadas
        if df.shape[1] < 2:
            print(f"⚠️ {name}: archivo no tiene datos suficientes")
            continue

        # Convertir fecha si es necesario
        if not pd.api.types.is_datetime64_any_dtype(df["date"]):
            df["date"] = pd.to_datetime(df["date"], errors="coerce")

        df = df.dropna(subset=["date"])
        df["date"] = df["date"].dt.strftime("%Y-%m-%d")  # Formato ISO
        df = df[["date", df.columns[1]]].rename(columns={df.columns[1]: name})

        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on="date", how="outer")

        station_names.append(name_clean)

    except Exception as e:
        print(f"⚠️ Error procesando {name}: {e}")
        errores.append(name)

# === 3. Generar archivo con coordenadas como cabecera ===
if merged_df is not None:
    merged_df = merged_df.sort_values("date")

    # Asegurar que el orden de las columnas corresponde a station_names
    columns_order = ["date"]
    lats, lons = [], []

    for f in all_files:
        station = os.path.basename(f).replace(".csv", "")
        station_clean = unidecode.unidecode(station.strip().upper())

        if station_clean in station_names:
            columns_order.append(station)
            lat, lon = coord_dict[station_clean]
            lats.append(round(lat, 4))
            lons.append(round(lon, 4))

    merged_df = merged_df[columns_order]

    # Construir cabeceras
    header_lat = ["N"] + lats
    header_lon = ["E"] + lons

    # Crear archivo final
    output_path = "D:/S/SENAMHI PERU/IDWstations.csv"
    with open(output_path, "w", encoding="utf-8", newline='') as f:
        f.write(",".join(map(str, header_lat)) + "\n")
        f.write(",".join(map(str, header_lon)) + "\n")
        merged_df.to_csv(f, index=False, header=False, float_format="%.1f")  # Solo 1 decimal

    print("✅ Archivo final guardado como 'IDWstations.csv'")

else:
    print("❌ No se pudo crear el archivo final.")

# Mostrar errores si los hay
if errores:
    print("\nErrores en estas estaciones:")
    print(", ".join(errores))


✅ Archivo final guardado como 'IDWstations.csv'


In [17]:
############################################ file to IDW Interpolation (R) ###############################################
import pandas as pd
import os
from glob import glob
import unidecode
from datetime import timedelta

# === 1. Cargar coordenadas ===
coord_df = pd.read_csv("D:/S/SENAMHI PERU/Stations Senamhi.csv", encoding="latin1")
coord_df["station_clean"] = coord_df["Stations"].str.strip().str.upper().apply(unidecode.unidecode)
coord_df = coord_df.sort_values("station_clean").reset_index(drop=True)

# Diccionario de coordenadas
coord_dict = dict(zip(coord_df["station_clean"], zip(coord_df["Lon"], coord_df["Lat"])))

# === 2. Leer archivos de estaciones ===
folder = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO"
files = glob(os.path.join(folder, "*.csv"))

# Fechas deseadas
start_date = pd.to_datetime("1981-01-01")
end_date = pd.to_datetime("1983-12-31")

# === 3. Procesar y agrupar por fecha ===
station_data = []

for file in files:
    station = os.path.basename(file).replace(".csv", "")
    station_clean = unidecode.unidecode(station.strip().upper())

    if station_clean not in coord_dict:
        print(f"⚠️ Estación {station} no está en coordenadas.")
        continue

    try:
        df = pd.read_csv(file)

        if "date" not in df.columns or "prec" not in df.columns:
            print(f"⚠️ {station}: columnas 'date' o 'prec' no están presentes.")
            continue

        df["date"] = pd.to_datetime(df["date"], dayfirst=False, errors="coerce")
        df = df.dropna(subset=["date"])
        df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]

        lon, lat = coord_dict[station_clean]

        for _, row in df.iterrows():
            current_date = row["date"]
            next_date = current_date + timedelta(days=1)
            station_data.append({
                "lon": lon,
                "lat": lat,
                "date": current_date,
                "end_date": next_date,
                "prcp": row["prec"]
            })

    except Exception as e:
        print(f"⚠️ Error procesando {station}: {e}")

# === 4. Agrupar por fecha y asignar sp.ID del 1 al N ===
df_all = pd.DataFrame(station_data)
df_all = df_all.sort_values(by=["date", "lat", "lon"]).reset_index(drop=True)
df_all["sp.ID"] = df_all.groupby("date").cumcount() + 1  # reinicia el ID cada fecha

# Renombrar columnas
df_all.rename(columns={
    "date": "time",
    "end_date": "endTime"
}, inplace=True)

# Convertir fechas a formato MM/DD/YYYY
df_all["time"] = df_all["time"].dt.strftime("%m/%d/%Y")
df_all["endTime"] = df_all["endTime"].dt.strftime("%m/%d/%Y")

# polars
pl_df = pl.from_pandas(df_all)

# .parquet
output_path = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO/stfdf_prec.parquet"
pl_df.write_parquet(output_path)

# === 5. Guardar resultado ===
#output_path = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO/stfdf_prec.csv"
#df_all.to_csv(output_path, index=False)
print("✅ Archivo generado:", output_path)


⚠️ Estación stfdf_prec no está en coordenadas.
✅ Archivo generado: D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO/stfdf_prec.parquet


In [24]:
import pandas as pd ################################## ramdom forest ##################################################
import os
from glob import glob
import unidecode
from datetime import timedelta

# === 1. Cargar coordenadas y datos estáticos ===
coord_df = pd.read_csv("D:/S/SENAMHI PERU/Stations Senamhi.csv", encoding="latin1")
coord_df["station_clean"] = coord_df["Stations"].str.strip().str.upper().apply(unidecode.unidecode)
coord_df = coord_df.sort_values("station_clean").reset_index(drop=True)

# Añadir staid (ID por estación)
#coord_df["staid"] = range(1, len(coord_df) + 1)

# Diccionario con todas las variables
coord_dict = coord_df.set_index("station_clean").to_dict(orient="index")

# === 2. Leer archivos de estaciones ===
folder = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO"
files = glob(os.path.join(folder, "*.csv"))

# Fechas deseadas
start_date = pd.to_datetime("1981-01-01")
end_date = pd.to_datetime("1983-12-31")

# === 3. Procesar y agrupar por fecha ===
station_data = []

for file in files:
    station = os.path.basename(file).replace(".csv", "")
    station_clean = unidecode.unidecode(station.strip().upper())

    if station_clean not in coord_dict:
        print(f"⚠️ Estación {station} no está en coordenadas.")
        continue

    try:
        df = pd.read_csv(file)

        if "date" not in df.columns or "prec" not in df.columns:
            print(f"⚠️ {station}: columnas 'date' o 'prec' no están presentes.")
            continue

        df["date"] = pd.to_datetime(df["date"], errors="coerce")
        df = df.dropna(subset=["date"])
        df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]

        # Datos estáticos de la estación
        info = coord_dict[station_clean]
        lon, lat = info["Lon"], info["Lat"]

        for _, row in df.iterrows():
            current_date = row["date"]
            next_date = current_date + timedelta(days=1)

            station_data.append({
                "lon": lon,
                "lat": lat,
                "time": current_date,
                "endTime": next_date,
                "prcp": row["prec"],
                "tmax": row.get("tmax", None),
                "tmin": row.get("tmin", None),
                #"staid": info["staid"],
                #"h": info["sin"],        # si 'h' es sin, ajustar si no
                "dem": info["dem"],
                "twi": info["twi"],
                "sin": info["sin"],
                "cos": info["cos"]
            })

    except Exception as e:
        print(f"⚠️ Error procesando {station}: {e}")

# === 4. Crear DataFrame final ===
df_all = pd.DataFrame(station_data)
df_all = df_all.sort_values(by=["time", "lat", "lon"]).reset_index(drop=True)

# sp.ID reinicia cada fecha
df_all["sp.ID"] = df_all.groupby("time").cumcount() + 1

# timeIndex (contador de días desde start_date)
df_all["timeIndex"] = (df_all["time"] - start_date).dt.days + 1

# Formato de fechas
df_all["time"] = df_all["time"].dt.strftime("%m/%d/%Y")
df_all["endTime"] = df_all["endTime"].dt.strftime("%m/%d/%Y")

# === 5. Ordenar columnas ===
column_order = [
    "lon", "lat", "sp.ID", "time", "endTime", "timeIndex",
    "dem", "twi", "prcp", "tmax", "tmin", "sin", "cos"
]
df_all = df_all[column_order]
# === 5. Guardar ===
#output_path = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO/stfdf_pt_puno.csv"
#df_all.to_csv(output_path, index=False, encoding="utf-8")
print("✅ Archivo generado:", output_path)


✅ Archivo generado: D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO/stfdf_pt_puno.csv


In [27]:
import polars as pl

# Convertir el DataFrame final de pandas a polars
df_all_pl = pl.from_pandas(df_all)

# Guardar en formato Parquet
output_path_parquet = "D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO/stfdf_pt_puno.parquet"
df_all_pl.write_parquet(output_path_parquet)

print("✅ Archivo Parquet generado:", output_path_parquet)


✅ Archivo Parquet generado: D:/S/OUTPUT_CSV_SENAMHI_NC_SELECTED_PUNO/stfdf_pt_puno.parquet


In [9]:
!pip install pyarrow
